In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, window

# Create SparkSession
spark = SparkSession.builder \
    .appName("WordCountStreaming") \
    .getOrCreate()

# Define the schema for the streaming data
schema = "value STRING"

# Read streaming data from a TCP socket
lines = spark.readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"))

# Count the occurrences of each word
word_counts = words.groupBy("word").count()

# Output the word counts to the console
query = word_counts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

# Wait for the streaming to finish
query.awaitTermination()

# Stop the SparkSession
spark.stop()


24/05/12 13:32:49 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
24/05/12 13:32:49 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-2ad32b4d-36ff-4a40-860a-853fd210d761. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/05/12 13:32:49 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
| ...|    1|
|name|    1|
|  is|    1|
|  My|    1|
+----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
| ...|    1|
|name|    1|
|  is|    1|
|  My|    1|
|  hi|    1|
+----+-----+

